# Úkol č. 3 - Segmentace zákazníků e-shopu (do 29. listopadu)

Jednou z důležitých aplikací shlukování je **segmentace zákazníků** (angl. **customer segmentation**). 

Předpokládejme, že máme následující obchodní údaje o prodejích (resp. nákupech z pohledu zákazníků):
TransactionID - ID nákupu,
CustomerID - ID zákazníka, 
Date - datum nákupu, 
Total - celková cena nákupu.

Chceme najít segmenty zákazníků, kteří se chovají podobně. K tomu je dobré informace z jednotlivých nákupů pro individuální zákazníky agregovat. Tj. získat pro každého zákazníka jeden řádek.

Populárním přístupem je **RFM**, což znamená:

- **R**ecency: Počet dnů od posledního nákupu (poslední datum v datasetu pro daného zákazníka).
    - Počet dnů počítejte ke dni uskutečnění poslendní transakce v celém datasetu (tj. 12/19/2015), nikoli k dnešku. Tváříme se, že jde o aktuální data.
- **F**requency: Počet nákupů. Občas se vynechávají zákazníci s jediným nákupem. Pro jednoduchost je zde ale necháme.
- **M**onetary: Celková suma, kterou daný zákazník utratil.

## Zdroj dat
Budeme pracovat s daty z jednoho (skoro) vymyšleného eshopu:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("eshop.csv")

In [ ]:
df.head(10)

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
- Vytvořte `rfm` data frame, kde každý řádek odpovídá jednomu zákazníkovi a sloupce (příznaky) jsou uvedené výše.
- Pomocí algoritmu `K-means` proveďte shlukování. Nějakým způsobem také odhadněte nejlepší počet shluků (podrobně vysvětlete).
- Zabývejte se vlivem přeškálování dat (standardizace příznaků). Tj. určete, zda je přeškálování vhodné, a proveďte ho.
- Interpretujte jednotlivé shluky. Použijte získané shluky k odlišení "superstar" zákazníků (vysoká monetary, vysoká frequency a nízká recency) od nezajímavých  zákazníků (vysoká recency, nízká frequency, nízká monetary).

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
- (až +4 body) Proveďte analýzu vytvořených shluků pomocí metody silhouette (https://en.wikipedia.org/wiki/Silhouette_(clustering)).
- (až +4 body) Zkuste provést to samé s modifikovanou verzí **RFM**, kde Recency = "maximum počtu měsíců od posledního nákupu a čísla 1", Frequency = "maximum počtu nákupů daného zákazníka v posledních 12 měsících a čísla 1", Monetary = "Nejvyšší hodnota nákupu daného zákazníka". Porovnejte s původním přístupem.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte Jupyter Notebook.
  * Ke komentování toho, co v notebooku děláte, použijte Markdown buňky.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni


## Řešení
### Vytvoření RFM dataframe

In [ ]:
from dateutil.parser import parse

# parse date from string
df["Date"] = df["Date"].apply(lambda x: parse(x))
df["ID"] = pd.Series(range(0, df.shape[0]))

# set last date
last_date = max(df["Date"])

# calculate RFM values
rfm = df.groupby('Customer ID').agg({'Date' : lambda x: (last_date - x.max()).days,
                                     'ID' : 'count', 
                                     'Subtotal' : 'sum'})

# rename the columns
rfm = rfm.rename(columns = {'Date' : 'Recency', 
                            'ID' : 'Frequency', 
                            'Subtotal' : 'Monetary'})

### Smazání outlierů
V dataframu se nachází jeden výjimečný případ zákazníka, který má velmi vysokou monetary oproti ostatním. Zbavíme se ho.

In [ ]:
print(rfm.loc[4912])
rfm = rfm.drop(4912)

In [ ]:
rfm.head(20)

### Histogramy příznaků před standardizací

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(14, 12))
rfm["Recency"].hist(ax=axes[0], bins=50).set_xlabel("Recency")
rfm["Frequency"].hist(ax=axes[1], bins=50).set_xlabel("Frequency")
rfm["Monetary"].hist(ax=axes[2], bins=50).set_xlabel("Monetary")

### Standardizace

Jak můžeme vidět, tak histogramy všech tří příznaků mají kladný koeficient šikmosti (mají tzv. pravý ocas). Proto by bylo vhodné je nějak transformovat do stavu, kdy budou mít koeficient šikmosti co nejblíže nule, a budou se podobat normálnímu rozdělení. Toto půjde krásně udělat u "Recency" a "Monetary". Bohužel u "Frequency" je minimální hodnota 1, která je ještě k tomu zastoupená v drtivé většině. Proto transformujeme pouze příznaky "Recency" a "Monetary". Využijeme PowerTransformer z scikitu, který provádí "chytrou" (umí rozhodnout, kterou funkci použít na transformaci) transformaci a zároveň umí i škálovat příznaky. Škálovat hodnoty u příznaku "Frequency" nebudeme, dá nám to větší důraz tomuto příznaku a k-means bude více rozdělovat podle frekvence.

#### Power transform
https://en.wikipedia.org/wiki/Power_transform

yeo-johnson je v podstatě box-cox doplněn o vstupy s negativními hodnotami


In [ ]:
from sklearn.preprocessing import PowerTransformer

pt = PowerTransformer(copy=True, method='yeo-johnson', standardize=True)
rfm_transformed = pd.DataFrame(pt.fit_transform(rfm[["Recency", "Monetary"]]),
                               index=rfm[["Recency", "Monetary"]].index,
                               columns=rfm[["Recency", "Monetary"]].columns)

rfm_transformed["Frequency"] = rfm["Frequency"].copy()

# pt = PowerTransformer(copy=True, method='yeo-johnson', standardize=True)
# rfm_transformed = pd.DataFrame(pt.fit_transform(rfm),
#                                index=rfm.index,
#                                columns=rfm.columns)

### Histogramy příznaků po standardizaci

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(14, 12))
rfm_transformed["Recency"].hist(ax=axes[0], bins=50).set_xlabel("Recency")
rfm_transformed["Frequency"].hist(ax=axes[1], bins=50).set_xlabel("Frequency")
rfm_transformed["Monetary"].hist(ax=axes[2], bins=50).set_xlabel("Monetary")

### K-Means
#### Zjištění nejlepší hodnoty počtu shluků pro standardizovaný dataframe
Obecně rozhodnout o nějlepším počtu shluků je problematické. Využijeme "elbow" metodu.

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns

wcss = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters= k, init= "k-means++")
    kmeans.fit(rfm_transformed)
    wcss[k] = kmeans.inertia_

sns.pointplot(x = list(wcss.keys()), y = list(wcss.values()))
plt.xlabel("Clusters")
plt.ylabel("WCSS")
plt.show()

#### K-Means pro standardizovaný dataframe, vizualizace
Zvolíme hodnotu 5 pro počet shluků. 

In [ ]:
import plotly
plotly.offline.init_notebook_mode()

def plot_kmeans(df, description="plot"):
    default_colors = [
        "#1f77b4",  # muted blue
        "#ff7f0e",  # safety orange
        "#2ca02c",  # cooked asparagus green
        "#d62728",  # brick red
        "#9467bd",  # muted purple
        "#ffff00",  # yellow
        "#e377c2",  # raspberry yogurt pink
        "#7f7f7f",  # middle gray
        "#8c564b",  # chestnut brown
        "#17becf"   # blue-teal
    ]
    
    def get_scatters(clusters):
        scatters = []
        for i, cluster in zip(range(0, len(clusters)), clusters):
            scatters.append(dict(mode = "markers",
                            name = "Cluster " + str(i+1),
                            type = "scatter3d",    
                            x = cluster.values[:,0], y = cluster.values[:,1], z = cluster.values[:,2],
                            marker = dict( size=2, color=default_colors[i])))
        return scatters

    scatters = get_scatters([df.loc[df["Cluster"] == x] for x in range(0, df["Cluster"].nunique())])
    layout = dict(title = description,
                  scene = dict(xaxis = dict(zeroline=True),
                               yaxis = dict(zeroline=True),
                               zaxis = dict(zeroline=True),
                               xaxis_title="Recency",
                               yaxis_title="Frequency",
                               zaxis_title="Monetary")
                 )
    plotly.offline.iplot(dict(data=scatters, layout=layout), filename="mesh3d_sample")

km = KMeans(n_clusters=5, init= "k-means++", random_state=42)
km.fit(rfm_transformed)
rfm_with_pt = rfm.copy()
rfm_with_pt["Cluster"] = km.labels_
    
plot_kmeans(rfm_with_pt, description="Visualization of clusters with power transform")
# rfm_with_pt[rfm_with_pt['Cluster']]

#### K-Means pro dataframe bez standardizace, vizualizace
Pro porovnání s dataframem, kde jsme neprovedli standardizaci. Je vidět, že algoritmus bere velice v potaz příznak "Recency" a snaží se rozdělovat podle něj.

In [ ]:
km = KMeans(n_clusters=5, init="k-means++", random_state=42)
km.fit(rfm)
rfm["Cluster"] = km.labels_

plot_kmeans(rfm, description="Visualization of clusters without power transform")

### Modifikovaná verze RFM
- (až +4 body) Zkuste provést to samé s modifikovanou verzí **RFM**, kde Recency = "maximum počtu měsíců od posledního nákupu a čísla 1", Frequency = "maximum počtu nákupů daného zákazníka v posledních 12 měsících a čísla 1", Monetary = "Nejvyšší hodnota nákupu daného zákazníka". Porovnejte s původním přístupem.

In [ ]:
from dateutil.relativedelta import relativedelta

def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

# calculate RM values
rfm_modified = df.groupby('Customer ID').agg({'Date' : lambda x: max(1, diff_month(last_date, x.max())),
                                              'Subtotal' : 'max'})

# rename the columns
rfm_modified = rfm_modified.rename(columns = {'Date' : 'Recency',
                                              'Subtotal' : 'Monetary'})

# calculate F values
freq = df[df["Date"] > last_date - relativedelta(years=1)].groupby('Customer ID').agg({'ID' : "count"})

from dateutil.relativedelta import relativedelta

def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

# calculate RM values
rfm_modified = df.groupby('Customer ID').agg({'Date' : lambda x: max(1, diff_month(last_date, x.max())),
                                              'Subtotal' : 'max'})

# rename the columns
rfm_modified = rfm_modified.rename(columns = {'Date' : 'Recency',
                                              'Subtotal' : 'Monetary'})

# calculate F values
freq = df[df["Date"] > last_date - relativedelta(years=1)].groupby('Customer ID').agg({'ID' : "count"})

# add F to RM
rfm_modified["Frequency"] = freq["ID"]
rfm_modified["Frequency"] = rfm_modified["Frequency"].fillna(1)

# power transform
pt = PowerTransformer(copy=True, method='yeo-johnson', standardize=True)
rfm_modified_transformed = pd.DataFrame(pt.fit_transform(rfm_modified[["Recency", "Monetary"]]),
                               index=rfm_modified[["Recency", "Monetary"]].index,
                               columns=rfm_modified[["Recency", "Monetary"]].columns)

rfm_modified_transformed["Frequency"] = rfm_modified["Frequency"].copy()

km = KMeans(n_clusters=5, init="k-means++", random_state=42)
km.fit(rfm_modified_transformed)
rfm_modified["Cluster"] = km.labels_

plot_kmeans(rfm_modified, description="Visualization of modified RFM")